In [1]:
#-*- coding:utf-8 -*-
from commonTool import *
import glob


running on Windows


In [ ]:
#################################################
# EVALUATE-PREPARE.xml 실행되어야 함.

In [7]:
# 기준일 로딩

bf = open(resourceDir + 'BASIS-DAYS.txt', 'r', encoding='utf-8')

# 0.YYYYMM, 1.FIRST_DAY, 2.SECOND_DAY, 3.PREV_DAY, 4.LAST_DAY, 5.NEXT_DAY
lineText = bf.readline() # 제목

basisDays = []
quaterBasisDays = []

while True:
    lineText = bf.readline()
    if not lineText:
        break
        
    lineText = lineText.strip()
    if len(lineText) <= 0:
        continue

    dayData = lineText.split('\t')
    
    if dayData[0][0:4] == '2012' or dayData[0] == '201301': continue # 데이터 부족
        
    basisDays.append([dayData[0], dayData[3], dayData[4]])
    
    if dayData[0] > '202001':
        quaterBasisDays.append([dayData[0], dayData[3], dayData[4]])

bf.close()


In [3]:
# 로직 적용한 결과 정리

def makeAnalysisReport(resultFolder, initialCash, period, feeRate = 0.015, withProcess = True):
    statResult = [f for f in glob.glob(resultFolder + '*-STAT.txt')]
    
    if len(statResult) == 0:
        print('no result file')
        return
    
    statResult.sort()
    
    m = 0
    y = 0
    cash = initialCash
    maxV = initialCash
    maxMdd = 0
    cagr = 0
    
    if withProcess:
        print('\t'.join(['Year', '매수일', '매도일', '손익', '투자금', '수익률', 'CAGR', 'MDD']))
    
    for statPath in statResult:
        m += period
        
        sf = open(statPath, 'r', encoding='utf-8')
        lineText = sf.readline() # 제목: --> 매수일, 매도일, R_SUM
        lineText = sf.readline() # 결과
        
        rec = lineText.split('\t')
        profitLoss = int(rec[2])

        newCash = round((cash + profitLoss) * (1.0 - feeRate))

        maxV = max(maxV, newCash)
        if newCash < cash:
            mdd = (newCash - maxV) / maxV
            maxMdd = min(mdd, maxMdd)

        y = m / 12
        cagr = pow(newCash / initialCash, 1 / y) - 1
        cagr = round(cagr * 100, 2)
        
        if withProcess:
            print('\t'.join([str(y), rec[0], rec[1],
                format(profitLoss, ',d').rjust(12), format(newCash, ',d').rjust(15),
                str(round(profitLoss / cash * 100, 2)).rjust(9), str(cagr).rjust(9), str(round(maxMdd * -100, 2))]))

        sf.close()
        cash = newCash

    if withProcess: print('')
    print('Year:', y, '  Invest:', format(initialCash, ',d'), '  Cash:', format(cash, ',d'),
        '  CAGR:', cagr, '%', '  Max MDD:', round(maxMdd * -100, 2), '%\n\n')


In [7]:
testResult = [
    '4PXX-25-10T-10-M2',
    '5PXR-25-10T-10-M2',
    # '4PXX-25-10T-15-M2',
    # '4PXX-25-10T-20-M2',
    'END' # 콤마 편하게 하려고 넣었음
]

for key in testResult:
    if key == 'END': break
        
    print('#', key)
    makeAnalysisReport(outputRawPath + 'logicResult' + os.path.sep + key + os.path.sep, 10000000,
                       (3 if key.endswith('-P3') else 6), 0.015, True)


# 4PXX-25-10T-10-M2
Year: 8.5   Invest: 10,000,000   Cash: 131,481,824   CAGR: 35.4 %   Max MDD: 6.49 %


# 5PXR-25-10T-10-M2
Year: 8.5   Invest: 10,000,000   Cash: 111,727,486   CAGR: 32.84 %   Max MDD: 3.91 %




In [ ]:
testResult = [
    '4PXX-25-10T-10-M2',
    '4PXX-25-10T-10-M3',
    '4PXX-25-10T-10-M4',
    '4PXX-25-10T-10-M5',
    '4PXX-25-10T-10-M6',
    'END' # 콤마 편하게 하려고 넣었음
]

for key in testResult:
    if key == 'END': break
        
    print('#', key)
    makeAnalysisReport(outputRawPath + 'logicResult' + os.path.sep + key + os.path.sep, 10000000, 6, 0.015, True)


In [10]:
testResult = [
    '4PXX-25-10T-10-M2',
    '4PXX-25-10T-20-M2',
    'END' # 콤마 편하게 하려고 넣었음
]

for key in testResult:
    if key == 'END': break
        
    print('#', key)
    makeAnalysisReport(outputRawPath + 'logicResult' + os.path.sep + key + os.path.sep, 10000000, 6, 0.015, True)


# 4PXX-25-10T-10-M2
Year	매수일	매도일	손익	투자금	수익률	CAGR	MDD
0.5	20130228	20130830	   1,804,682	     11,627,612	    18.05	     35.2	0
1.0	20130830	20140228	     943,980	     12,383,018	     8.12	    23.83	0
1.5	20140228	20140829	   3,306,281	     15,453,960	     26.7	    33.67	0
2.0	20140829	20150227	    -782,621	     14,451,269	    -5.06	    20.21	6.49
2.5	20150227	20150831	   1,184,610	     15,401,341	      8.2	    18.86	6.49
3.0	20150831	20160229	    -296,476	     14,878,292	    -1.93	    14.16	6.49
3.5	20160229	20160831	   4,340,573	     18,930,582	    29.17	     20.0	6.49
4.0	20160831	20170228	   8,858,211	     27,371,961	    46.79	    28.63	6.49
4.5	20170228	20170831	   4,512,004	     31,405,706	    16.48	    28.96	6.49
5.0	20170831	20180228	   7,799,345	     38,616,975	    24.83	    31.03	6.49
5.5	20180228	20180831	   9,382,634	     47,279,615	     24.3	    32.64	6.49
6.0	20180831	20190228	    -800,345	     45,782,081	    -1.69	    28.86	6.49
6.5	20190228	20190830	   6,917,824	     51,9

In [11]:
testResult = [
    # '4PXX-25-10T-20-M2',
    'Q-4PXX-25-10T-10-M2',
    'Q-RND-25-10T-10-M2',
    'END' # 콤마 편하게 하려고 넣었음
]

for key in testResult:
    if key == 'END': break
        
    print('#', key)
    makeAnalysisReport(outputRawPath + 'logicResult' + os.path.sep + key + os.path.sep, 10000000, 3, 0.015, False)


# Q-4PXX-25-10T-10-M2
Year: 1.5   Invest: 10,000,000   Cash: 27,212,360   CAGR: 94.91 %   Max MDD: 0 %


# Q-RND-25-10T-10-M2
Year: 1.5   Invest: 10,000,000   Cash: 26,008,637   CAGR: 89.12 %   Max MDD: 0 %




In [ ]:
makeAnalysisReport(outputRawPath + 'logicResult' + os.path.sep + '4PXX-25-10T-10-M2' + os.path.sep, 10000000, 6, 0.015, True)

In [4]:
analId = '4PXX'  # 로직명: 4PXX, MAGIC, %PXR, RND
calcDate = yyyymm() # 로직 실행할 대상 월 '202107'

# sys.argv[0] 은 python 파일

if len(sys.argv) > 1:
    analId = sys.argv[1]
    
if len(sys.argv) > 2:
    calcDate = sys.argv[2]

print('running', analId, calcDate)

cash = 10000000  # 초기 투자금
period = 6  # 리밸런싱 기간 (월)
buyCount = 20  # 매수 종목 개수
totalAmount = 25 # 25 # 시가총액 하위 퍼센트 조건
volumeLimit = 10000 # 10000 # 거래량 조건
startMonth = 2

analTitle = analId + '-' + str(totalAmount) + '-' + str(round(volumeLimit / 1000)) + 'T-' + str(buyCount) \
    + '-M' + str(startMonth) + ('-P' + str(period) if period != 6 else '')

# 로직 스크립트
scriptPathName = crawlegoScriptPath + 'ANAL-' + analId + '.xml'

# 결과 저장 위치
resultFolder = outputRawPath + 'logicResult' + os.path.sep + analTitle + os.path.sep
# [os.remove(f) for f in glob.glob(resultFolder + '*.txt')]
mkdir(resultFolder)


done = False
for basis in basisDays:
    if basis[0] != calcDate: continue
    
    parameter = {
        'BASIS_DAY': basis[1],
        'BUY_DAY': basis[2],
        'SELL_DAY': basis[2],
        'CASH': str(cash),
        'TOP_N': str(buyCount),
        'PERIOD': 'ANNUAL',
        'IN_PATH': outputRawPath + 'index' + os.path.sep + (os.path.sep if runServerType == 1 else ''),
        'OUT_PATH': resultFolder + (os.path.sep if runServerType == 1 else ''),
        'TOTAL_AMOUNT': str(totalAmount),
        'VOL_LIMIT': str(volumeLimit),
        'DO_SERVER': '13.124.29.70'
    }
    
    print(basis, 'running...')
    retCode = runDashScript(scriptPathName, parameter, False)
    print(basis, 'returns', retCode)
    print('result folder:', resultFolder)
    done = True
    break

if not done:
    print('invalid argument', analId, calcDate)

running -f C:\Users\admin\AppData\Roaming\jupyter\runtime\kernel-ca37cb5f-c52e-4825-9c17-45549015769b.json
invalid argument -f C:\Users\admin\AppData\Roaming\jupyter\runtime\kernel-ca37cb5f-c52e-4825-9c17-45549015769b.json


In [13]:
len(sys.argv)

3

In [9]:
# 년간 재무 데이터를 이용한 분석

analId = '4PXX'  # 로직명: 4PXX, MAGIC, %PXR, RND
inital_cash = 10000000  # 초기 투자금
period = 6  # 리밸런싱 기간 (월)
buyCount = 10  # 매수 종목 개수
totalAmount = 25 # 25 # 시가총액 하위 퍼센트 조건
volumeLimit = 10000 # 10000 # 거래량 조건

startMonth = 2

analTitle = analId + '-' + str(totalAmount) + '-' + str(round(volumeLimit / 1000)) + 'T-' + str(buyCount) \
    + '-M' + str(startMonth) + ('-P' + str(period) if period != 6 else '')

print('running', analTitle)


feeRate = 0.015 # 1.5% 수수료

# 로직 스크립트
scriptPathName = crawlegoScriptPath + 'ANAL-' + analId + '.xml'

# 결과 저장 위치
resultFolder = outputRawPath + 'logicResult' + os.path.sep + analTitle + os.path.sep
# [os.remove(f) for f in glob.glob(resultFolder + '*.txt')]
mkdir(resultFolder)

# 계산 시작
m = 0
cash = inital_cash

for i in range(startMonth - 2, len(basisDays), period):
    basis = basisDays[i]
    sellIdx = min(i + period, len(basisDays) - 1)
    
    parameter = {
        'BASIS_DAY': basis[1],
        'BUY_DAY': basis[2],
        'SELL_DAY': basisDays[sellIdx][2],
        'CASH': str(cash),
        'TOP_N': str(buyCount),
        'PERIOD': 'ANNUAL',
        'IN_PATH': outputRawPath + 'index' + os.path.sep + (os.path.sep if runServerType == 1 else ''),
        'OUT_PATH': resultFolder + (os.path.sep if runServerType == 1 else ''),
        'TOTAL_AMOUNT': str(totalAmount),
        'VOL_LIMIT': str(volumeLimit),
        'DO_SERVER': '13.124.29.70'
    }

    m += period
    retCode = runDashScript(scriptPathName, parameter, False)
    
    print(m / 12, basis[0], basis[2], basisDays[sellIdx][2], 'ret code', retCode)
    
    if retCode != 0:
        continue

    # 결과 정리
    statPath = resultFolder + basis[1] + '-STAT.txt'
    sf = open(statPath, 'r', encoding='utf-8')
    
    lineText = sf.readline() # 제목: 매수일, 매도일, R_SUM
    lineText = sf.readline() # 결과
    
    rec = lineText.split('\t')
    profitLoss = int(rec[2])
    sf.close()
    
    cash = round((cash + profitLoss) * (1.0 - feeRate))
    # end of for

makeAnalysisReport(resultFolder, inital_cash, period, feeRate, True)
# end of script

running 4PXX-25-10T-10-M2
0.5 201302 20130228 20130830 ret code 0
1.0 201308 20130830 20140228 ret code 0
1.5 201402 20140228 20140829 ret code 0
2.0 201408 20140829 20150227 ret code 0
2.5 201502 20150227 20150831 ret code 0
3.0 201508 20150831 20160229 ret code 0
3.5 201602 20160229 20160831 ret code 0
4.0 201608 20160831 20170228 ret code 0
4.5 201702 20170228 20170831 ret code 0
5.0 201708 20170831 20180228 ret code 0
5.5 201802 20180228 20180831 ret code 0
6.0 201808 20180831 20190228 ret code 0
6.5 201902 20190228 20190830 ret code 0
7.0 201908 20190830 20200228 ret code 0
7.5 202002 20200228 20200831 ret code 0
8.0 202008 20200831 20210226 ret code 0
8.5 202102 20210226 20210831 ret code 0
9.0 202108 20210831 20210831 ret code 0
Year	매수일	매도일	손익	투자금	수익률	CAGR	MDD
0.5	20130228	20130830	   1,804,682	     11,627,612	    18.05	     35.2	0
1.0	20130830	20140228	     943,980	     12,383,018	     8.12	    23.83	0
1.5	20140228	20140829	   3,306,281	     15,453,960	     26.7	    33.67	0
2.

In [12]:
# 분기 재무 데이터를 이용한 분석

analId = 'RND'  # 로직명
inital_cash = 10000000  # 초기 투자금
period = 3  # 리밸런싱 기간 (월)
buyCount = 10  # 매수 종목 개수
totalAmount = 200 # 25 # 시가총액 하위 퍼센트 조건
volumeLimit = 1 # 10000 # 거래량 조건

startMonth = 2

analTitle = 'Q-' + analId + '-' + str(totalAmount) + '-' + str(round(volumeLimit / 1000)) + 'T-' + str(buyCount) + '-M' + str(startMonth)

print('running', analTitle)


feeRate = 0.015 # 1.5% 수수료

# 로직 스크립트
scriptPathName = crawlegoScriptPath + 'ANAL-' + analId + '.xml'

# 결과 저장 위치
resultFolder = outputRawPath + 'logicResult' + os.path.sep + analTitle + os.path.sep
# [os.remove(f) for f in glob.glob(resultFolder + '*.txt')]
mkdir(resultFolder)

# 계산 시작
m = 0
cash = inital_cash

for i in range(startMonth - 2, len(quaterBasisDays), period):
    basis = quaterBasisDays[i]
    sellIdx = min(i + period, len(quaterBasisDays) - 1)
    
    parameter = {
        'BASIS_DAY': basis[1],
        'BUY_DAY': basis[2],
        'SELL_DAY': quaterBasisDays[sellIdx][2],
        'CASH': str(cash),
        'TOP_N': str(buyCount),
        'PERIOD': 'QUARTER',
        'IN_PATH': outputRawPath + 'index' + os.path.sep + (os.path.sep if runServerType == 1 else ''),
        'OUT_PATH': resultFolder + (os.path.sep if runServerType == 1 else ''),
        'TOTAL_AMOUNT': str(totalAmount),
        'VOL_LIMIT': str(volumeLimit),
        'DO_SERVER': '13.124.29.70'
    }

    m += period
    retCode = runDashScript(scriptPathName, parameter, False)
    
    print(m / 12, basis[0], 'ret code', retCode)
    
    if retCode != 0:
        continue

    # 결과 정리
    statPath = resultFolder + basis[1] + '-STAT.txt'
    sf = open(statPath, 'r', encoding='utf-8')
    
    lineText = sf.readline() # 제목: 매수일, 매도일, R_SUM
    lineText = sf.readline() # 결과
    
    rec = lineText.split('\t')
    profitLoss = int(rec[2])
    sf.close()
    
    cash = round((cash + profitLoss) * (1.0 - feeRate))
    # end of for

makeAnalysisReport(resultFolder, inital_cash, period, feeRate, True)
# end of script

running Q-RND-200-0T-10-M2
0.25 202002 ret code 0
0.5 202005 ret code 0
0.75 202008 ret code 0
1.0 202011 ret code 0
1.25 202102 ret code 0
1.5 202105 ret code 0
Year	매수일	매도일	손익	투자금	수익률	CAGR	MDD
0.25	20200228	20200529	     118,250	      9,966,476	     1.18	    -1.33	0.34
0.5	20200529	20200831	   1,554,742	     11,348,400	     15.6	    28.79	0.34
0.75	20200831	20201130	   1,289,805	     12,448,632	    11.37	    33.91	0.34
1.0	20201130	20210226	     173,114	     12,432,420	     1.39	    24.32	0.34
1.25	20210226	20210531	   2,228,600	     14,441,105	    17.93	    34.18	0.34
1.5	20210531	20210713	     577,770	     14,793,592	      4.0	    29.83	0.34

Year: 1.5   Invest: 10,000,000   Cash: 14,793,592   CAGR: 29.83 %   Max MDD: 0.34 %




In [5]:
resultFolder = outputRawPath + 'logicResult' + os.path.sep + '4PXX-25-10T-10-M2' + os.path.sep
saveFolder = temporaryPath + 'analMM' + os.path.sep

scriptPathName = crawlegoScriptPath + 'CALC-MOMENTUM.xml'

mmCount = 3
analResult = [f for f in glob.glob(resultFolder + '*-RESULT.txt')]

for selFile in analResult:
    p = selFile.rfind(os.path.sep)
    basisDay = selFile[p+1:p+9]
    
    parameter = {
        'BASIS_DAY': basisDay,
        'IN_PATH': resultFolder + (os.path.sep if runServerType == 1 else ''),
        'OUT_PATH': saveFolder + (os.path.sep if runServerType == 1 else ''),
        'MM_COUNT': str(mmCount),
        'DO_SERVER': '13.124.29.70'
    }

    retCode = runDashScript(scriptPathName, parameter, False)
    print(basisDay, 'returns', retCode)
    

20130227 returns 0


In [19]:
resultFolder = outputRawPath + 'logicResult' + os.path.sep + '4PXX-25-10T-10-M2' + os.path.sep
saveFolder = temporaryPath + 'analMM' + os.path.sep

scriptPathName = crawlegoScriptPath + 'CALC-AVGINDEX.xml'

mmCount = 3
analResult = [f for f in glob.glob(resultFolder + '*-RESULT.txt')]

for selFile in analResult:
    p = selFile.rfind(os.path.sep)
    basisDay = selFile[p+1:p+9]
    
    parameter = {
        'BASIS_DAY': basisDay,
        'IN_PATH': resultFolder + (os.path.sep if runServerType == 1 else ''),
        'OUT_PATH': saveFolder + (os.path.sep if runServerType == 1 else ''),
        'DO_SERVER': '13.124.29.70'
    }

    retCode = runDashScript(scriptPathName, parameter, False)
    print(basisDay, 'returns', retCode)
    

20130227 returns 0
20130829 returns 0
20140227 returns 0
20140828 returns 0
20150226 returns 0
20150828 returns 0
20160226 returns 0
20160830 returns 0
20170227 returns 0
20170830 returns 0
20180227 returns 0
20180830 returns 0
20190227 returns 0
20190829 returns 0
20200227 returns 0
20200828 returns 0
20210225 returns 0


In [7]:
DD = [
    ['20130227', '20130228', '20130731'],
    ['20130829', '20130930', '20131129'],
    ['20140227', '20140228', '20140829'],
    ['20140828', '20140828', '20140828'],
    ['20150226', '20150227', '20150630'],
    ['20150828', '20150828', '20150828'],
    ['20160226', '20160229', '20160831'],
    ['20160830', '20160831', '20170228'],
    ['20170227', '20170331', '20170831'],
    ['20170830', '20171130', '20180228'],
    ['20180227', '20180228', '20180731'],
    ['20180830', '20180830', '20180830'],
    ['20190227', '20190228', '20190830'],
    ['20190829', '20191031', '20200228'],
    ['20200227', '20200228', '20200831'],
    ['20200828', '20201030', '20210226'],
    ['20210225', '20210226', '20210621']
]

resultFolder = outputRawPath + 'logicResult' + os.path.sep + '4PXX-25-10T-10-M2' + os.path.sep
saveFolder = 'D:\work\jupyter\krx\intermediate\profit' + os.path.sep

scriptPathName = crawlegoScriptPath + 'CALC-PROFIT.xml'

cash = 10000000
for day in DD:
    parameter = {
        'BASIS_DAY': day[0],
        'BUY_DAY': day[1],
        'SELL_DAY': day[2],
        'CASH': str(cash),
        'IN_PATH': resultFolder + (os.path.sep if runServerType == 1 else ''),
        'OUT_PATH': saveFolder + (os.path.sep if runServerType == 1 else ''),
        'DO_SERVER': '13.124.29.70'
    }

    retCode = runDashScript(scriptPathName, parameter, False)
    print(day[0], 'returns', retCode)

    sf = open(saveFolder + day[0] + '-STAT.txt', 'r', encoding='utf-8')
    lineText = sf.readline() # 제목: --> 매수일, 매도일, R_SUM
    lineText = sf.readline() # 결과

    rec = lineText.split('\t')
    profitLoss = int(rec[2])
    
    sf.close()

    cash += profitLoss
    print(day[0], profitLoss, cash)


20130227 returns 0
20130227 1676677 11676677
20130829 returns 0
20130829 -248092 11428585
20140227 returns 0
20140227 3065977 14494562
20140828 returns 0
20140828 -29023 14465539
20150226 returns 0
20150226 -812285 13653254
20150828 returns 0
20150828 97490 13750744
20160226 returns 0
20160226 4556302 18307046
20160830 returns 0
20160830 8537447 26844493
20170227 returns 0
20170227 -5007457 21837036
20170830 returns 0
20170830 1847390 23684426
20180227 returns 0
20180227 3787583 27472009
20180830 returns 0
20180830 16080 27488089
20190227 returns 0
20190227 3510661 30998750
20190829 returns 0
20190829 581888 31580638
20200227 returns 0
20200227 1233924 32814562
20200828 returns 0
20200828 14787355 47601917
20210225 returns 0
20210225 22997330 70599247


In [2]:
# 

resultFolder = outputRawPath + 'highlow' + os.path.sep
mkdir(resultFolder)

scriptPathName = crawlegoScriptPath + 'POC-MOMENTUM.xml'

midSizeComp = resourceDir + 'midSizeComp.txt'

mf = open(midSizeComp, 'r', encoding='utf-8')

while True:
    lineText = mf.readline()
    if not lineText:
        break
        
    lineText = lineText.strip()
    if len(lineText) <= 0:
        continue
        
    # lineText is company code
    
    parameter = {
        'COMP_CODE': lineText,
        'OUT_PATH': resultFolder,
        'DO_SERVER': '13.124.29.70'
    }

    retCode = runDashScript(scriptPathName, parameter, False)
    print(lineText, 'returns', retCode)
    break

mf.close()

000020 returns 0
